In [11]:
import urllib.request
import json
import codecs

Read event feed as JSON data format

In [3]:
with urllib.request.urlopen("https://calendar.tamu.edu/live/json/events/group") as url:
  data = json.load(url)

In [ ]:
print("data length:", len(data))
print(data[0].keys())

Create event dictionary

In [5]:
events = []
for item in data:
  event_dict = {}
  event_dict["title"] = item["title"]
  event_dict["group_title"] = item["group_title"]
  event_dict["url"] = item["url"]
  event_dict["date"] = item["date"]
  event_dict["date_ts"] = item["date_ts"]
  event_dict["date2_ts"] = item["date2_ts"]
  event_dict["event_types"] = item["event_types"]
  event_dict["event_types_audience"] = item["event_types_audience"]
  event_dict["event_types_campus"] = item["event_types_campus"]
  event_dict["is_online"] = item["is_online"]
  event_dict["online_type"] = item["online_type"]
  event_dict["has_registration"] = item["has_registration"]
  event_dict["location"] = item["location"]
  event_dict["location_latitude"] = item["location_latitude"]
  event_dict["location_longitude"] = item["location_longitude"]
  event_dict["custom_room_number"] = item["custom_room_number"] if "custom_room_number" in item else None
  event_dict["description"] = item["description"]
  event_dict["thumbnail"] = item["thumbnail"]
  event_dict["thumbnail:html"] = item["thumbnail:html"] if "thumbnail:html" in item else None
  events.append(event_dict)

In [6]:
events[0]

{'title': 'Aggies Invent: Sandia National Laboratories',
 'group_title': 'College of Engineering',
 'url': 'https://calendar.tamu.edu/engineering/event/326667-aggies-invent-sandia-national-laboratories',
 'date': 'September 14',
 'date_ts': 1726290000,
 'date2_ts': None,
 'event_types': ['Training &amp; Workshops'],
 'event_types_audience': ['Researcher', 'Students'],
 'event_types_campus': ['Bryan-College Station'],
 'is_online': None,
 'online_type': None,
 'has_registration': None,
 'location': 'Zachry Engineering Education Complex',
 'location_latitude': 30.621087,
 'location_longitude': '-96.340498',
 'custom_room_number': 415,
 'description': '<h2 class="headline-group">\n  <span class="head">Solve Problems in 48 Hours.</span>\n</h2>',
 'thumbnail': 'https://calendar.tamu.edu/live/image/gid/83/width/80/height/80/crop/1/src_region/0,0,3200,1801/13380_24_0341_EEP_AI_AdMan_1000x563.jpg',
 'thumbnail:html': '<picture>\n\t<source type="image/webp" srcset="https://calendar.tamu.edu/liv

Prompt Engineering (Instead of manually can we do it dynamically?)

In [7]:
prompts = [
  "What is " + events[0]["title"] + "?",
  "Where is " + events[0]["title"] + " located?",
  "Who can attend " + events[0]["title"] + "?",
]

prompts

['What is Aggies Invent: Sandia National Laboratories?',
 'Where is Aggies Invent: Sandia National Laboratories located?',
 'Who can attend Aggies Invent: Sandia National Laboratories?']

Response Engineering

In [8]:
responses = [
  events[0]["title"] + " is hosted by " + events[0]["group_title"] + " and will take place at " + events[0]["location"] + " on " + events[0]["date"],
  "The event will take place at " + events[0]["location"] + ", Room " + str(events[0]["custom_room_number"]) + ", in " + events[0]["event_types_campus"][0],
  "The event is open to " + " and ".join(events[0]["event_types_audience"]),
]

responses

['Aggies Invent: Sandia National Laboratories is hosted by College of Engineering and will take place at Zachry Engineering Education Complex on September 14',
 'The event will take place at Zachry Engineering Education Complex, Room 415, in Bryan-College Station',
 'The event is open to Researcher and Students']

In [9]:
def generate_prompts_responses(event):
    title = event['title']
    location = event['location']
    room = event.get('custom_room_number', '')
    date = event['date']

    prompts_responses = []

    # Where is the event located?
    prompt = f"Where is {title} located?"
    response = f"The event will take place at {location}, Room {room}."
    prompts_responses.append((prompt, response))

    # When is the event happening?
    prompt = f"When is {title} happening?"
    response = f"{title} is scheduled for {date}."
    prompts_responses.append((prompt, response))

    # Who can attend the event?
    if event['event_types_audience'] is not None:
        audience = ', '.join(event['event_types_audience'])
        prompt = f"Who can attend {title}?"
        response = f"{title} is open to {audience}."
        prompts_responses.append((prompt, response))

    # What type of event is it?
    if event['event_types'] is not None:
        event_type = ', '.join(event['event_types'])
        prompt = f"What type of event is {title}?"
        response = f"{title} is classified as a {event_type}."
        prompts_responses.append((prompt, response))

    return prompts_responses

In [10]:
event_prompts_responses = generate_prompts_responses(events[0])

# Print the generated prompts and responses
for prompt, response in event_prompts_responses:
    print(f"Prompt: {prompt}")
    print(f"Response: {response}")
    print()

Prompt: Where is Aggies Invent: Sandia National Laboratories located?
Response: The event will take place at Zachry Engineering Education Complex, Room 415.

Prompt: When is Aggies Invent: Sandia National Laboratories happening?
Response: Aggies Invent: Sandia National Laboratories is scheduled for September 14.

Prompt: Who can attend Aggies Invent: Sandia National Laboratories?
Response: Aggies Invent: Sandia National Laboratories is open to Researcher, Students.

Prompt: What type of event is Aggies Invent: Sandia National Laboratories?
Response: Aggies Invent: Sandia National Laboratories is classified as a Training &amp; Workshops.



In [13]:
file = codecs.open('dynamic_event_data.txt', "w", "utf-8")
for event in events:
  prompts_responses = generate_prompts_responses(event)
  for prompt, response in prompts_responses:
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Response: {response}\n\n")
file.close()